In [91]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import compose, pipeline
from sklearn.model_selection import KFold
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import roc_auc_score, make_scorer, r2_score
from pandas import DataFrame, Series
import numpy as np
kf = KFold(n_splits=5, shuffle=True, random_state=42)
import xgboost as xgb
from xgboost import XGBRegressor
import optuna
from optuna import Trial
from optuna import create_study
from sklearn import compose
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
import plotly.express as px
#website help from 
#https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451

In [92]:
df = pd.read_csv('dataframes/df_normal_quality.csv', index_col=0)
y = df.SalePrice
X = df.drop(['PID', 'SalePrice'], axis =1).copy()

In [93]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
#from category_encoders import WOEEncoder

def instantiate_ordinal_encoder(trial: Trial)-> OrdinalEncoder:
    params = {
        'handle_unknown': "use_encoded_value", 
        'unknown_value': -1
    }
    
    return OrdinalEncoder(**params)

# def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
#     params = {
#         'handle_unknown': 'ignore',
#         'drop': trial.suggest_categorical('drop', [None, 'first'])
#     }
    
#     return OneHotEncoder(**params)

def instantiate_onehot_encoder(trial: Trial)-> OneHotEncoder:
    params = {
        'handle_unknown': 'ignore',
        'drop': None
    }
    
    return OneHotEncoder(**params)
    
Encoder = (
    OrdinalEncoder |
    OneHotEncoder 
    )

def instantiate_encoder (trial : Trial) -> Encoder:
    encoding_method = trial.suggest_categorical(
        'encoding_method', ['ordinal', 'onehot'])
    if encoding_method =='ordinal':
        encoder = instantiate_ordinal_encoder(trial)
    elif encoding_method =='onehot':
        encoder = instantiate_onehot_encoder(trial)
    
    return encoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

Scaler = (
  StandardScaler |
  MinMaxScaler |
  MaxAbsScaler |
  RobustScaler
)

def instantiate_scaler(trial : Trial) -> Scaler:
    method = trial.suggest_categorical(
    'scaling_method', ['standard', 'minmax', 'maxabs', 'robust']
    )
    if method=='standard':
        scaler = StandardScaler()
    elif method=='minmax':
        scaler = MinMaxScaler()
    elif method=='maxabs':
        scaler = MaxAbsScaler()
    elif method=='robust':
        scaler = RobustScaler()
        
    return scaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

def instantiate_processor(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_encoder(trial)
   # numerical_pipeline = StandardScaler()
    #categorical_pipeline = OneHotEncoder(handle_unknown="ignore", drop='first')
    
#     numerical_pipeline = instantiate_numerical_pipeline(trial)
#     categorical_pipeline = instantiate_categorical_pipeline(trial)
    
    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

def instantiate_processor_ordinal(trial : Trial, 
                          numerical_columns : list[str], 
                          categorical_columns : list[str]) -> ColumnTransformer:
    
    numerical_pipeline = instantiate_scaler(trial)
    categorical_pipeline = instantiate_ordinal_encoder(trial)

    processor = ColumnTransformer([
        ('numerical_pipeline', numerical_pipeline, numerical_columns),
        ('categorical_pipeline', categorical_pipeline, categorical_columns)
    ])
    return processor

# Lasso

In [94]:
def instantiate_lasso(trial : Trial) -> Lasso:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True)
    }

    return Lasso(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_lasso(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_lasso(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [95]:
from optuna import create_study

study_lasso = create_study(study_name='optimization', direction='maximize')

study_lasso.optimize(lambda trial: objective_lasso(trial, X, y), n_trials=200)

[I 2024-07-29 07:36:55,196] A new study created in memory with name: optimization
[I 2024-07-29 07:36:55,834] Trial 0 finished with value: 0.9413486527275285 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'alpha': 0.00023563714171645237}. Best is trial 0 with value: 0.9413486527275285.
[I 2024-07-29 07:36:56,217] Trial 1 finished with value: 0.9414753887038929 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'alpha': 0.0005712083490140479}. Best is trial 1 with value: 0.9414753887038929.
[I 2024-07-29 07:36:56,864] Trial 2 finished with value: 0.9416260982286614 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.00015033206075715856}. Best is trial 2 with value: 0.9416260982286614.
[I 2024-07-29 07:36:57,329] Trial 3 finished with value: 0.9351474416179325 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'alpha': 0.000635742639219267}. Best is trial 2 with value: 0.941

[I 2024-07-29 07:37:20,446] Trial 16 finished with value: 0.949983994475259 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.0003670720125247345}. Best is trial 14 with value: 0.9502615635493765.
[I 2024-07-29 07:37:21,485] Trial 17 finished with value: 0.9499302415130618 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.00037356477055176335}. Best is trial 14 with value: 0.9502615635493765.
[I 2024-07-29 07:37:22,444] Trial 18 finished with value: 0.9496683378651666 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.0004004399640285355}. Best is trial 14 with value: 0.9502615635493765.
[I 2024-07-29 07:37:23,140] Trial 19 finished with value: 0.9433426432139065 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'alpha': 0.0009808086891770458}. Best is trial 14 with value: 0.9502615635493765.
[I 2024-07-29 07:37:24,538] Trial 20 finished with value: 0.95063277

[I 2024-07-29 07:37:53,553] Trial 52 finished with value: 0.9508119046631849 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003092640694000414}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:37:54,316] Trial 53 finished with value: 0.9508114774099967 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.000299231237129812}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:37:55,104] Trial 54 finished with value: 0.9508112444206546 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00029865399528757763}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:37:55,783] Trial 55 finished with value: 0.9506341800396925 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0004502291261042328}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:37:56,920] Trial 56 finished with value: 0.95020364

[I 2024-07-29 07:38:21,142] Trial 81 finished with value: 0.9508112532801274 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.000298181469857414}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:38:21,863] Trial 82 finished with value: 0.9508089338819549 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00033007919435104166}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:38:22,762] Trial 83 finished with value: 0.950471698734787 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00019629900575499358}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:38:23,551] Trial 84 finished with value: 0.950804821824148 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00028857385361033425}. Best is trial 45 with value: 0.9508123262966481.
[I 2024-07-29 07:38:24,899] Trial 85 finished with value: 0.94968171

[I 2024-07-29 07:38:50,082] Trial 116 finished with value: 0.9481422448810559 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'alpha': 0.00016314465646387878}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:38:51,081] Trial 117 finished with value: 0.950570935807413 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00021842996446886577}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:38:52,166] Trial 118 finished with value: 0.9504568119712223 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00019266678797332076}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:38:53,014] Trial 119 finished with value: 0.9507891929882291 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003545303935855875}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:38:53,817] Trial 120 finished with value:

[I 2024-07-29 07:39:16,714] Trial 147 finished with value: 0.9507436120464379 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.000258674298424698}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:17,412] Trial 148 finished with value: 0.9502891469102662 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'alpha': 0.0002949119235740743}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:18,151] Trial 149 finished with value: 0.9507884285385062 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00035505472432560887}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:18,653] Trial 150 finished with value: 0.9418090204768192 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.00042688426036030194}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:19,356] Trial 151 finished with valu

[I 2024-07-29 07:39:41,753] Trial 176 finished with value: 0.9495105885331947 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0008059364203045801}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:42,542] Trial 177 finished with value: 0.9508017611328132 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.00034197258571487673}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:43,343] Trial 178 finished with value: 0.9508102087236308 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0002958618218254628}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:43,928] Trial 179 finished with value: 0.9417190057513846 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'alpha': 0.00023856272080633823}. Best is trial 102 with value: 0.9508125881814135.
[I 2024-07-29 07:39:44,562] Trial 180 finished with value

In [96]:
print(study_lasso.best_value)
print(study_lasso.best_params)

0.9508125881814135
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'alpha': 0.0003061236655610336}


In [97]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_lasso.get_trials()[trial].number)
    score.append(study_lasso.get_trials()[trial].value)
    parameters.append(list(study_lasso.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_lasso = pd.DataFrame.from_dict(optuna_dict)
optuna_lasso

,Trial,Score,Parameters
0,0,0.941349,"[(scaling_method, standard), (encoding_method, ordinal), (alpha, 0.00023563714171645237)]"
1,1,0.941475,"[(scaling_method, standard), (encoding_method, ordinal), (alpha, 0.0005712083490140479)]"
2,2,0.941626,"[(scaling_method, robust), (encoding_method, ordinal), (alpha, 0.00015033206075715856)]"
3,3,0.935147,"[(scaling_method, maxabs), (encoding_method, ordinal), (alpha, 0.000635742639219267)]"
4,4,0.941152,"[(scaling_method, robust), (encoding_method, ordinal), (alpha, 2.6641266594799548e-05)]"
5,5,0.949807,"[(scaling_method, standard), (encoding_method, onehot), (alpha, 0.00011328026394192677)]"
6,6,0.948423,"[(scaling_method, robust), (encoding_method, onehot), (alpha, 4.037586729565322e-05)]"
7,7,0.937837,"[(scaling_method, maxabs), (encoding_method, ordinal), (alpha, 0.00015270683963769906)]"
8,8,0.948032,"[(scaling_method, maxabs), (encoding_method, onehot), (alpha, 0.0002685650895205734)]"
9,9,0.948730,"[(scaling_method, minmax), (encoding_method, onehot), (alpha, 3.4834705940221466e-05)]"


In [15]:
pd.set_option('display.max_colwidth', None) 
optuna_lasso.Parameters

0                   [(scaling_method, minmax), (encoding_method, ordinal), (alpha, 1.6999779739443475e-05)]
1                 [(scaling_method, standard), (encoding_method, ordinal), (alpha, 0.00037901740871573876)]
2      [(scaling_method, robust), (encoding_method, onehot), (drop, first), (alpha, 0.0009834045353742098)]
3      [(scaling_method, minmax), (encoding_method, onehot), (drop, None), (alpha, 2.5666965172816898e-05)]
4                   [(scaling_method, maxabs), (encoding_method, ordinal), (alpha, 1.0571302084059589e-05)]
                                                       ...                                                 
195     [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.0003814762162935689)]
196    [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.00031881502123266083)]
197     [(scaling_method, robust), (encoding_method, onehot), (drop, None), (alpha, 0.0002681604864862338)]
198    [(scaling_method, rob

In [18]:
 optuna_lasso['alpha'] = optuna_lasso.Parameters.apply(lambda x: x[-1][1])

In [23]:
 optuna_lasso['encoding'] = optuna_lasso.Parameters.apply(lambda x: x[1][1])

In [24]:
optuna_lasso['scaling'] = optuna_lasso.Parameters.apply(lambda x: x[0][1])

In [25]:
optuna_lasso.to_csv('dataframes/optuna/optuna_lasso.csv')

In [35]:
fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna.png")

In [36]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'scaling',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_scaling.png")

In [37]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'alpha',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_alpha.png")

In [38]:

fig = px.scatter(optuna_lasso,
                 x="Trial", 
                 y="Score", 
                 hover_data=['Parameters'],
                 color = 'encoding',
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'Lasso tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9508126</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

fig.write_image("images/lasso_optuna_encoding.png")

# Ridge

In [98]:
def instantiate_ridge(trial : Trial) -> Ridge:
    params = {
        "alpha": trial.suggest_float("alpha", 1, 100, log=True)
    }

    return Ridge(**params)


def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ridge(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ridge(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [43]:
from optuna import create_study

study_ridge = create_study(study_name='optimization', direction='maximize')

#study_ridge.optimize(lambda trial: objective_ridge(trial, X, y), n_trials=20)

[I 2024-07-11 09:50:20,011] A new study created in memory with name: optimization


In [17]:
print(study_ridge.best_value)
print(study_ridge.best_params)

0.9496764273611051
{'scaling_method': 'standard', 'encoding_method': 'onehot', 'drop': None, 'alpha': 24.817075515507273}


# EN

In [99]:
def instantiate_en(trial : Trial) -> ElasticNet:
    params = {
        "alpha": trial.suggest_float("alpha", .00001, .001, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', .1, .7, log = True)
    }

    return ElasticNet(**params)



def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_en(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_en(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [40]:
from optuna import create_study

study_en = create_study(study_name='optimization', direction='maximize')

#study_en.optimize(lambda trial: objective_en(trial, X, y), n_trials=200)

[I 2024-07-11 09:50:19,990] A new study created in memory with name: optimization


In [66]:
# after adding in the L1 value
print(study_en.best_value)
print(study_en.best_params)

0.9508214083737423
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006921322276640134, 'l1_ratio': 0.4415555448483296}


In [29]:
print(study_en.best_value)
print(study_en.best_params)

0.950820633330468
{'scaling_method': 'robust', 'encoding_method': 'onehot', 'drop': None, 'alpha': 0.0006178621404820522}


# XGB

In [107]:
def instantiate_xgb(trial : Trial) -> XGBRegressor:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", .001, .1, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 4),
        "subsample": trial.suggest_float("subsample", 0.1, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5)
    }

    return XGBRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_xgb(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_xgb(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [108]:
from optuna import create_study

study_xgb = create_study(study_name='optimization', direction='maximize')

study_xgb.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=200)

[I 2024-07-29 19:18:59,365] A new study created in memory with name: optimization
[I 2024-07-29 19:19:01,336] Trial 0 finished with value: -1.9720493123224387 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'ordinal', 'n_estimators': 446, 'learning_rate': 0.005869981543671911, 'max_depth': 3, 'subsample': 0.20323911417731688, 'colsample_bytree': 0.5040274630503353, 'min_child_weight': 1}. Best is trial 0 with value: -1.9720493123224387.
[I 2024-07-29 19:19:08,029] Trial 1 finished with value: -0.15579172747555026 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'n_estimators': 948, 'learning_rate': 0.0035677656269343973, 'max_depth': 3, 'subsample': 0.3245234890610799, 'colsample_bytree': 0.9096041174541876, 'min_child_weight': 5}. Best is trial 1 with value: -0.15579172747555026.
[I 2024-07-29 19:19:08,975] Trial 2 finished with value: 0.9367650450876482 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'n_estimators'

[I 2024-07-29 19:22:42,811] Trial 22 finished with value: 0.9483549684443205 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 926, 'learning_rate': 0.054481562210723894, 'max_depth': 2, 'subsample': 0.46132473192212636, 'colsample_bytree': 0.8961932020264917, 'min_child_weight': 4}. Best is trial 16 with value: 0.9510810222907871.
[I 2024-07-29 19:22:57,861] Trial 23 finished with value: 0.9487750682492039 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 637, 'learning_rate': 0.029777645693304725, 'max_depth': 4, 'subsample': 0.6994656647466124, 'colsample_bytree': 0.9428279856508588, 'min_child_weight': 5}. Best is trial 16 with value: 0.9510810222907871.
[I 2024-07-29 19:23:13,141] Trial 24 finished with value: 0.9498412987219448 and parameters: {'scaling_method': 'maxabs', 'encoding_method': 'onehot', 'n_estimators': 808, 'learning_rate': 0.06282595631254716, 'max_depth': 3, 'subsample': 0.595353249863

[I 2024-07-29 19:27:19,228] Trial 44 finished with value: 0.9470520038685327 and parameters: {'scaling_method': 'robust', 'encoding_method': 'onehot', 'n_estimators': 728, 'learning_rate': 0.06918546178790552, 'max_depth': 3, 'subsample': 0.7318738523587515, 'colsample_bytree': 0.782041143672853, 'min_child_weight': 2}. Best is trial 16 with value: 0.9510810222907871.
[I 2024-07-29 19:27:21,929] Trial 45 finished with value: 0.9327996399453383 and parameters: {'scaling_method': 'robust', 'encoding_method': 'ordinal', 'n_estimators': 179, 'learning_rate': 0.0521280944121063, 'max_depth': 3, 'subsample': 0.8153989773625517, 'colsample_bytree': 0.8290064018727762, 'min_child_weight': 3}. Best is trial 16 with value: 0.9510810222907871.
[I 2024-07-29 19:27:30,203] Trial 46 finished with value: 0.9457797086521724 and parameters: {'scaling_method': 'minmax', 'encoding_method': 'onehot', 'n_estimators': 469, 'learning_rate': 0.039288072852519874, 'max_depth': 3, 'subsample': 0.632938355683119

[I 2024-07-29 19:32:27,451] Trial 66 finished with value: 0.9516199664666674 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 999, 'learning_rate': 0.04211299626055344, 'max_depth': 3, 'subsample': 0.4980483092394806, 'colsample_bytree': 0.9937724302280114, 'min_child_weight': 2}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:32:47,209] Trial 67 finished with value: 0.9510320498590481 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 992, 'learning_rate': 0.032242812766106015, 'max_depth': 3, 'subsample': 0.4552214080493642, 'colsample_bytree': 0.9847757390695335, 'min_child_weight': 2}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:33:05,471] Trial 68 finished with value: 0.9442174832938882 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 961, 'learning_rate': 0.013995050979304478, 'max_depth': 3, 'subsample': 0.510664769

[I 2024-07-29 19:38:22,462] Trial 88 finished with value: 0.9517088290094762 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 989, 'learning_rate': 0.03577714924318214, 'max_depth': 3, 'subsample': 0.5594199838475186, 'colsample_bytree': 0.9024744578155284, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:38:34,181] Trial 89 finished with value: 0.9501220597836418 and parameters: {'scaling_method': 'standard', 'encoding_method': 'ordinal', 'n_estimators': 883, 'learning_rate': 0.035801105935934774, 'max_depth': 3, 'subsample': 0.5508151378916272, 'colsample_bytree': 0.8937761211271278, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:38:41,122] Trial 90 finished with value: 0.9391874384914887 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 348, 'learning_rate': 0.03275568449421139, 'max_depth': 3, 'subsample': 0.613534910

[I 2024-07-29 19:44:10,407] Trial 110 finished with value: 0.9509571270647762 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 876, 'learning_rate': 0.048759219345812246, 'max_depth': 3, 'subsample': 0.4541479947568875, 'colsample_bytree': 0.9986061000175864, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:44:26,524] Trial 111 finished with value: 0.9518441657743117 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 848, 'learning_rate': 0.061648837774991805, 'max_depth': 3, 'subsample': 0.5913292031666039, 'colsample_bytree': 0.9483606256769012, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:44:36,931] Trial 112 finished with value: -6.554510159408477 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 840, 'learning_rate': 0.0010592871083305022, 'max_depth': 3, 'subsample': 0.6338

[I 2024-07-29 19:49:24,974] Trial 132 finished with value: 0.9505211234777309 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 802, 'learning_rate': 0.03875569995196243, 'max_depth': 3, 'subsample': 0.4574697194836935, 'colsample_bytree': 0.7338441340711962, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:49:41,368] Trial 133 finished with value: 0.9506203616697847 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 830, 'learning_rate': 0.08355757235420343, 'max_depth': 3, 'subsample': 0.5441741505315424, 'colsample_bytree': 0.9571159725202943, 'min_child_weight': 5}. Best is trial 51 with value: 0.952923611621055.
[I 2024-07-29 19:49:57,528] Trial 134 finished with value: 0.9523263079736817 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 903, 'learning_rate': 0.04601828138395837, 'max_depth': 3, 'subsample': 0.48047847

[I 2024-07-29 19:55:20,435] Trial 154 finished with value: 0.9528040258216377 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 899, 'learning_rate': 0.051454712455778796, 'max_depth': 3, 'subsample': 0.4134416785820464, 'colsample_bytree': 0.9793026249087157, 'min_child_weight': 2}. Best is trial 149 with value: 0.9534381314534116.
[I 2024-07-29 19:55:38,077] Trial 155 finished with value: 0.9511838058079987 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 913, 'learning_rate': 0.05405840745977514, 'max_depth': 3, 'subsample': 0.40988638650096515, 'colsample_bytree': 0.9870374291266046, 'min_child_weight': 2}. Best is trial 149 with value: 0.9534381314534116.
[I 2024-07-29 19:55:55,703] Trial 156 finished with value: 0.9498327866320825 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 929, 'learning_rate': 0.07126536172812326, 'max_depth': 3, 'subsample': 0.38

[I 2024-07-29 20:01:27,587] Trial 176 finished with value: 0.9511461666549905 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 968, 'learning_rate': 0.04081417394308537, 'max_depth': 3, 'subsample': 0.4835414616245649, 'colsample_bytree': 0.5590607902934845, 'min_child_weight': 2}. Best is trial 149 with value: 0.9534381314534116.
[I 2024-07-29 20:01:45,125] Trial 177 finished with value: 0.9522810980700374 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 951, 'learning_rate': 0.04761118384117342, 'max_depth': 3, 'subsample': 0.5279084132473046, 'colsample_bytree': 0.7032816319577867, 'min_child_weight': 2}. Best is trial 149 with value: 0.9534381314534116.
[I 2024-07-29 20:02:01,558] Trial 178 finished with value: 0.951625064481189 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 978, 'learning_rate': 0.037544008712061705, 'max_depth': 3, 'subsample': 0.5821

[I 2024-07-29 20:07:33,273] Trial 198 finished with value: 0.9523130328291856 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 927, 'learning_rate': 0.05011069991302511, 'max_depth': 3, 'subsample': 0.5938873255411979, 'colsample_bytree': 0.7216257265540275, 'min_child_weight': 2}. Best is trial 184 with value: 0.9535143024497366.
[I 2024-07-29 20:07:51,048] Trial 199 finished with value: 0.9499098957180125 and parameters: {'scaling_method': 'standard', 'encoding_method': 'onehot', 'n_estimators': 961, 'learning_rate': 0.07394670804480963, 'max_depth': 3, 'subsample': 0.5483599314461408, 'colsample_bytree': 0.7405014851747527, 'min_child_weight': 2}. Best is trial 184 with value: 0.9535143024497366.


In [106]:
print(study_xgb.best_value)
print(study_xgb.best_params)

0.8796063404360446
{'scaling_method': 'robust', 'learning_rate': 0.9959721609475207, 'loss': 'exponential', 'n_estimators': 848}


In [54]:
trial_number = []
score = []
parameters = []
optuna_dict = {}
trials = range(200)

for trial in trials:
    trial_number.append(study_xgb.get_trials()[trial].number)
    score.append(study_xgb.get_trials()[trial].value)
    parameters.append(list(study_xgb.get_trials()[trial].params.items()))

optuna_dict['Trial'] = trial_number
optuna_dict['Score'] = score
optuna_dict['Parameters'] = parameters

optuna_xgb = pd.DataFrame.from_dict(optuna_dict)
optuna_xgb

,Trial,Score,Parameters
0,0,0.250057,"[(scaling_method, standard), (encoding_method, ordinal), (n_estimators, 813), (learning_rate, 0.004546685209408476), (max_depth, 2), (subsample, 0.5972054222840305), (colsample_bytree, 0.8291820721150385), (min_child_weight, 5)]"
1,1,0.947215,"[(scaling_method, maxabs), (encoding_method, ordinal), (n_estimators, 487), (learning_rate, 0.07226798074248438), (max_depth, 4), (subsample, 0.39395707465472163), (colsample_bytree, 0.7667781923927004), (min_child_weight, 4)]"
2,2,0.935425,"[(scaling_method, robust), (encoding_method, ordinal), (n_estimators, 753), (learning_rate, 0.011094121360774055), (max_depth, 4), (subsample, 0.95069650705626), (colsample_bytree, 0.5885222277620357), (min_child_weight, 3)]"
3,3,0.940379,"[(scaling_method, maxabs), (encoding_method, onehot), (drop, first), (n_estimators, 840), (learning_rate, 0.018408191755019152), (max_depth, 3), (subsample, 0.9540745397031227), (colsample_bytree, 0.9137879206046258), (min_child_weight, 4)]"
4,4,0.905482,"[(scaling_method, minmax), (encoding_method, onehot), (drop, first), (n_estimators, 186), (learning_rate, 0.03678202054471029), (max_depth, 2), (subsample, 0.6746007767704554), (colsample_bytree, 0.665830441618052), (min_child_weight, 1)]"
...,...,...,...
195,195,0.951854,"[(scaling_method, standard), (encoding_method, onehot), (drop, first), (n_estimators, 981), (learning_rate, 0.06000902301833092), (max_depth, 3), (subsample, 0.5756228536210479), (colsample_bytree, 0.8834950350520847), (min_child_weight, 3)]"
196,196,0.952006,"[(scaling_method, standard), (encoding_method, ordinal), (n_estimators, 970), (learning_rate, 0.0512564879110385), (max_depth, 3), (subsample, 0.5325453973380234), (colsample_bytree, 0.9084335394685096), (min_child_weight, 3)]"
197,197,0.951308,"[(scaling_method, standard), (encoding_method, onehot), (drop, first), (n_estimators, 958), (learning_rate, 0.04122534118292556), (max_depth, 3), (subsample, 0.5374824724301858), (colsample_bytree, 0.861514903939569), (min_child_weight, 3)]"
198,198,0.952089,"[(scaling_method, standard), (encoding_method, onehot), (drop, first), (n_estimators, 998), (learning_rate, 0.045540108160620985), (max_depth, 3), (subsample, 0.5598844297848496), (colsample_bytree, 0.8847013623068011), (min_child_weight, 3)]"


In [56]:
optuna_xgb.to_csv('dataframes/optuna/optuna_xgb.csv')

In [ ]:
{'scaling_method': 'standard', 
 'encoding_method': 'onehot', 'drop': 'first', 
 'n_estimators': 987, 
 'learning_rate': 0.04571203871971512, 
 'max_depth': 3, 
 'subsample': 0.5029585844426339, 
 'colsample_bytree': 0.8889268050013618, 
 'min_child_weight': 3}


In [57]:
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_rows', None)
#optuna_xgb.sort_values(by='Score', ascending=False)

In [68]:
optuna_xgb['scaling'] =optuna_xgb.Parameters.apply(lambda x: x[0][1])
optuna_xgb['encoding'] = optuna_xgb.Parameters.apply(lambda x: x[1][1])
optuna_xgb['n_estimators'] = optuna_xgb.Parameters.apply(lambda x: x[-6][1])
optuna_xgb['learning_rate'] = optuna_xgb.Parameters.apply(lambda x: x[-5][1])
optuna_xgb['max_depth'] = optuna_xgb.Parameters.apply(lambda x: x[-4][1])
optuna_xgb['subsample'] = optuna_xgb.Parameters.apply(lambda x: x[-3][1])
optuna_xgb['colsample'] = optuna_xgb.Parameters.apply(lambda x: x[-2][1])
optuna_xgb['min_child'] = optuna_xgb.Parameters.apply(lambda x: x[-1][1])

In [85]:
fig = px.scatter(optuna_xgb.loc[optuna_xgb.Score > .93],
                 x="Trial", 
                 y="Score", 
                 color = 'learning_rate',
                 hover_data=['Parameters'],
                labels = {'Trial': 'Optuna Trial', 'Score': 'Score (R-Squared)'},
                title = 'XGB tuned with Optuna after 200 trials  <br><sup> Best Score: 0.9539</sup>')

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=8,
        font_family="Rockwell"
    )
)

fig.show()

# RF

In [109]:
def instantiate_rf(trial : Trial) -> RandomForestRegressor:
    params = {
    'bootstrap':trial.suggest_categorical('bootstrap', [True]),
    'n_estimators': trial.suggest_int('n_estimators', 250, 500),
    'max_depth': trial.suggest_int('max_depth', 50, 75),
    'min_samples_split': trial.suggest_int('min_samples_split', 4, 6),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
    }
    
    return RandomForestRegressor(**params)
  
def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_rf(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_rf(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [ ]:
from optuna import create_study

study_rf = create_study(study_name='optimization', direction='maximize')

study_rf.optimize(lambda trial: objective_rf(trial, X, y), n_trials=200)

[I 2024-07-29 20:07:51,083] A new study created in memory with name: optimization
[I 2024-07-29 20:09:25,692] Trial 0 finished with value: 0.9007273711541205 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 257, 'max_depth': 70, 'min_samples_split': 5, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9007273711541205.
[I 2024-07-29 20:12:21,385] Trial 1 finished with value: 0.909677377002513 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 325, 'max_depth': 67, 'min_samples_split': 4, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.909677377002513.
[I 2024-07-29 20:15:17,667] Trial 2 finished with value: 0.9067015853000758 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 395, 'max_depth': 70, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.909677377002513.
[I 2024-07-29 20:17:21,949] Trial 3 finished with value: 0.9073943959481742 and parameters: {'scali

[I 2024-07-29 21:36:32,238] Trial 30 finished with value: 0.9047636959222125 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 286, 'max_depth': 62, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 21:40:31,609] Trial 31 finished with value: 0.9087253257490395 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 389, 'max_depth': 67, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 21:44:47,475] Trial 32 finished with value: 0.9096298044259294 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 419, 'max_depth': 70, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 21:49:06,762] Trial 33 finished with value: 0.9080566146840162 and parameters: {'scaling_method': 'robust', 'bootstrap': True, 'n_estimators': 428, 'max_depth': 

[I 2024-07-29 23:18:25,200] Trial 60 finished with value: 0.909477499267789 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 377, 'max_depth': 60, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 23:22:52,306] Trial 61 finished with value: 0.9096790420746559 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 416, 'max_depth': 72, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 23:27:16,940] Trial 62 finished with value: 0.909267414001969 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 429, 'max_depth': 70, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 21 with value: 0.910629279373543.
[I 2024-07-29 23:31:46,732] Trial 63 finished with value: 0.9080736333700012 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 440, 'max_depth': 72

[I 2024-07-30 01:10:42,318] Trial 90 finished with value: 0.9093403137725729 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 396, 'max_depth': 70, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 01:14:47,350] Trial 91 finished with value: 0.908868646304564 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 406, 'max_depth': 72, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 01:19:03,508] Trial 92 finished with value: 0.9073039972624316 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 417, 'max_depth': 73, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 01:22:46,518] Trial 93 finished with value: 0.9068526390513423 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 369, 'max_dept

[I 2024-07-30 02:58:41,459] Trial 120 finished with value: 0.9079643798737886 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 423, 'max_depth': 66, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 03:02:59,655] Trial 121 finished with value: 0.9096445542882852 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 419, 'max_depth': 71, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 03:07:12,017] Trial 122 finished with value: 0.9089508344757365 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 411, 'max_depth': 71, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 03:11:30,181] Trial 123 finished with value: 0.9087542395825361 and parameters: {'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 426, 'max_d

[I 2024-07-30 04:56:22,790] Trial 150 finished with value: 0.9097415296371685 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 336, 'max_depth': 66, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 04:59:48,383] Trial 151 finished with value: 0.9100840004003323 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 347, 'max_depth': 66, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 05:03:11,845] Trial 152 finished with value: 0.9093945226202038 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 343, 'max_depth': 65, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 82 with value: 0.9122847254161676.
[I 2024-07-30 05:06:31,378] Trial 153 finished with value: 0.9092310790818945 and parameters: {'scaling_method': 'standard', 'bootstrap': True, 'n_estimators': 335

In [51]:
print(study_rf.best_value)
print(study_rf.best_params)

0.9106961840498735
{'scaling_method': 'robust', 'encoding_method': 'ordinal', 'bootstrap': True, 'n_estimators': 338, 'max_depth': 73, 'min_sample_split': 4, 'min_sample_leaf': 2}


# gbr

In [ ]:
def instantiate_gbr(trial : Trial) -> GradientBoostingRegressor:
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'max_depth': trial.suggest_int ('max_depth', 2, 6),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    'min_samples_split': trial.suggest_int ('min_sample_split', 6, 10),
    'min_samples_leaf': trial.suggest_int ('min_sample_leaf', 1, 10)
    }
    
    return GradientBoostingRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_gbr(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_gbr(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [ ]:
from optuna import create_study

study_gbr = create_study(study_name='optimization', direction='maximize')

study_gbr.optimize(lambda trial: objective_gbr(trial, X, y), n_trials=200)

In [35]:
print(study_gbr.best_value)
print(study_gbr.best_params)

0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_leaf': 10}


# Adaboost

In [ ]:
def instantiate_ada(trial : Trial) -> AdaBoostRegressor:
    params = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1, log=True),
    'loss': trial.suggest_categorical('loss',['linear', 'square', 'exponential']),
    'n_estimators': trial.suggest_int('n_estimators', 1, 1000)
    }
    
    return AdaBoostRegressor(**params)

def instantiate_model(trial : Trial, numerical_columns : list[str], 
                      categorical_columns : list[str]) -> Pipeline:
    
    processor = instantiate_processor_ordinal(
        trial, numerical_columns, categorical_columns
    )
    
    learner = instantiate_ada(trial)
    
    model_pipe = Pipeline([
    ('processor', processor),
    ('model', learner)
    ])
    
    model = compose.TransformedTargetRegressor(regressor= model_pipe,
                                                func=np.log, inverse_func=np.exp)
    
    return model

def objective_ada(trial : Trial, X : DataFrame,
              y : np.ndarray | Series, 
              numerical_columns : Optional[list[str]]=None, 
              categorical_columns : Optional[list[str]]=None, 
              random_state : int=42) -> float:
    
    if numerical_columns is None:
        numerical_columns = [
            *X.select_dtypes(exclude=['object', 'category']).columns
        ]
    
    if categorical_columns is None:
        categorical_columns = [
            *X.select_dtypes(include=['object', 'category']).columns
        ]
    
    model = instantiate_model(trial, numerical_columns, categorical_columns)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    r2 = make_scorer(r2_score)
    scores = cross_val_score(model, X, y, scoring= r2, cv=kf)
    
    return np.min([np.mean(scores), np.median([scores])])

In [ ]:
from optuna import create_study

study_ada = create_study(study_name='optimization', direction='maximize')

study_ada.optimize(lambda trial: objective_ada(trial, X, y), n_trials=200)

# Loop through?

In [61]:
studies = [study_lasso, study_ridge, study_en, study_xgb, study_rf, study_gbr, study_ada]
objectives = [objective_lasso, 
              objective_ridge, 
              objective_en,
              objective_xgb,
              objective_rf,
              objective_gbr,
              objective_ada]

# for study, objective in zip(studies, objectives):
#     study.optimize(lambda trial: objective(trial, X, y), n_trials=2)
    

[<function __main__.objective_lasso(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_ridge(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_en(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: Optional[list[str]] = None, categorical_columns: Optional[list[str]] = None, random_state: int = 42) -> float>,
 <function __main__.objective_xgb(trial: optuna.trial._trial.Trial, X: pandas.core.frame.DataFrame, y: numpy.ndarray | pandas.core.series.Series, numerical_columns: O

In [52]:
for study in studies:
    print (study.best_value)
    print (study.best_params)

0.8735606638813536
{'scaling_method': 'robust', 'learning_rate': 0.30065504792403025, 'loss': 'exponential', 'n_estimators': 428}
0.7980960053211354
{'scaling_method': 'robust', 'learning_rate': 0.006736329679908936, 'loss': 'exponential', 'n_estimators': 615}
0.8177296198440576
{'scaling_method': 'maxabs', 'learning_rate': 0.04083092875953665, 'loss': 'exponential', 'n_estimators': 196}
0.951622985121114
{'scaling_method': 'minmax', 'encoding_method': 'onehot', 'drop': None, 'n_estimators': 979, 'learning_rate': 0.046054720840163293, 'max_depth': 4, 'subsample': 0.8282908794962028, 'colsample_bytree': 0.5510975070017484, 'min_child_weight': 10}
0.9094102521730991
{'scaling_method': 'maxabs', 'bootstrap': True, 'n_estimators': 494, 'max_depth': 62, 'min_samples_split': 4, 'min_samples_leaf': 1}
0.9214824552885755
{'scaling_method': 'robust', 'n_estimators': 868, 'learning_rate': 0.005429634831521693, 'max_depth': 3, 'subsample': 0.8347093949409654, 'min_sample_split': 10, 'min_sample_l